In [3]:
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text
import numpy as np
mbti= pd.read_csv('../gonzaloalvarez/final_project/mbti_changed.csv')

In [4]:
mbti.head()

,type,posts,type_index,cleaned_text
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,enfp and intj moments sportscenter not t...
1,ENTP,'I'm finding the lack of me in these posts ver...,3,im finding the lack of me in these pos...
2,INTP,'Good one _____ https://www.youtube.com/wat...,11,good one of course to which i say i ...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,dear intp i enjoyed our conversation the...
4,ENTJ,'You're fired.|||That's another silly misconce...,2,youre fired thats another silly misconcep...


In [15]:
from sklearn.model_selection import train_test_split


train,test= train_test_split(mbti)
train,val = train_test_split(train)


print(train.shape)
print(test.shape)
print(val.shape)


(4879, 4)
(2169, 4)
(1627, 4)


In [9]:
def new_model():
    
    text_input = tf.keras.layers.Input(shape=[], dtype=tf.string, name='text')
    
    preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name='preprocessing')
    
    encoder_inputs = preprocessing_layer(text_input)
    
    encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",
    trainable=False, name='BERT_encoder')
    
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net= tf.keras.layers.Dense(64)(net)
    net = tf.keras.layers.Dropout(0.5)(net)
    net= tf.keras.layers.Dense(32)(net)
    net = tf.keras.layers.Dropout(0.5)(net)
    net = tf.keras.layers.Dense(16, activation='softmax', name='classifier')(net)
    
    
    return tf.keras.Model(text_input, net)

In [10]:
model= new_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'default': (None, 7 109482241   preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
____________________________________________________________________________________________

In [11]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = tf.metrics.CategoricalAccuracy()

model.compile(loss=loss, optimizer='adam', metrics=metrics)

In [16]:
train_array= np.array(train['cleaned_text'])
val_array= np.array(val['cleaned_text'])

In [17]:
one_hot_labels = tf.keras.utils.to_categorical(train.type_index.values, num_classes=16)
val_labels=tf.keras.utils.to_categorical(val.type_index.values, num_classes=16)

In [19]:
model.fit(train_array, one_hot_labels, validation_data=(val_array, val_labels), verbose=1, epochs=1)

153/153 [==============================] - 971s 6s/step - loss: 3.1931 - categorical_accuracy: 0.1300 - val_loss: 2.3493 - val_categorical_accuracy: 0.1598


In [20]:
test_hot_labels = tf.keras.utils.to_categorical(test.type_index.values, num_classes=16)
test_array= np.array(test['cleaned_text'])
results= model.evaluate(test_array, test_hot_labels, verbose=1)

68/68 [==============================] - 334s 5s/step - loss: 2.3343 - categorical_accuracy: 0.1614
